### 이력서 파일로 면접 질문 예상 프롬프트

In [2]:
# 생성한 이력서 파일로 면접 예상 질문 생성 (LangChain)

# 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")


OPENAI_API_KEY: sk


In [4]:
# 이력서 로딩 및 벡터 저장소 생성 
def load_resume_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    이력서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    """
    try:
        print(f"이력서 파일 로딩 중: {file_path}")

        # pdf나 txt 파일에 따라 문서 로드
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"이력서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e

In [10]:
# 면접 예상 질문 생성 
def generate_interview_questions(
    vectorstore,
    company_name: str = "",
    desired_job_role: str = "", # 새로 추가된 파라미터: 희망 직무
    interview_type: str = "general", # 'general', 'technical', 'behavioral'
    temperature: float = 0.7
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)와 파라미터를 기반으로 면접 예상 질문을 생성합니다.
    생성된 질문은 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 질문 유형에 따른 추가 지침
        type_instructions = {
            "general": "지원자의 전반적인 경험, 역량, 회사 적합성 등을 평가할 수 있는 종합적인 질문을 생성해주세요.",
            "technical": "지원자의 이력서에 언급된 기술 스택과 프로젝트 경험을 바탕으로 심도 있는 기술 면접 질문을 생성해주세요.",
            "behavioral": "지원자의 행동 양식, 문제 해결 능력, 팀워크, 리더십 등을 평가할 수 있는 인성 및 행동 기반 면접 질문을 생성해주세요."
        }
        instruction = type_instructions.get(interview_type, type_instructions["general"])

        # 회사 이름이 제공된 경우 추가 지침
        company_instruction = f"면접 볼 회사가 '{company_name}'이라면, 해당 회사의 비전, 제품, 문화와 연관된 질문을 1~2개 포함해주세요." if company_name else ""
        
        # 희망 직무가 제공된 경우 추가 지침
        job_role_instruction = ""
        if desired_job_role:
            job_role_instruction = f"지원자가 희망하는 직무는 '{desired_job_role}'입니다. 이 직무에 필요한 역량, 경험, 그리고 직무 관련 시나리오에 대한 질문을 1~2개 포함해주세요."
            if interview_type == "technical":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무의 핵심 기술 스택과 관련된 심층 기술 질문을 포함해주세요."
            elif interview_type == "behavioral":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무에서 발생할 수 있는 상황에 대한 행동 기반 질문을 포함해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접관이며, 제공된 이력서 내용을 바탕으로 지원자에게 면접 예상 질문을 생성해야 합니다.
        다음 지침에 따라 질문을 생성하고, 각 질문에 대한 간략한 답변 가이드라인도 함께 제공해주세요.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 약점 등을 파악하세요.
        2.  총 5~7개의 면접 질문을 생성해주세요.
        3.  질문 유형: {instruction} {company_instruction} {job_role_instruction}
        4.  각 질문에 대해 답변자가 어떤 점을 강조해야 하는지 1~2문장으로 간략한 '가이드라인'을 포함해주세요.
        5.  전반적인 답변 가이드라인을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "questions": [
            {{{{
            "question": "질문 내용 1",
            "guidance": "답변 가이드라인 1"
            }}}},
            {{{{
            "question": "질문 내용 2",
            "guidance": "답변 가이드라인 2"
            }}}}
        ],
        "overall_guidance": "전반적인 답변 가이드라인 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 면접 질문을 할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        # temperature 값 -> 질문의 다양성(창의성)에 영향을 줌
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "면접 질문을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        # LangChain의 create_retrieval_chain은 'answer' 키에 최종 응답을 반환
        # 그 응답은 LLM이 생성한 JSON 문자열이므로 파싱 필요
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"면접 질문 생성 중 오류 발생: {str(e)}")
        return {"error": f"면접 질문 생성 중 오류가 발생했습니다: {str(e)}"}


In [ ]:
# 제공하는 PDF 파일로 예상 면접 질문 생성 - 마케터 버전
resume_file_path = "./resume/예시 파일 (마케터).pdf"

# 파일 존재 여부 확인 (오류 방지용)
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성
        # 마케터 이력서이므로 'general' 또는 'behavioral' 유형이 적합할 수 있습니다.
        # 필요에 따라 company_name을 추가할 수 있습니다.
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="ABC 마케팅", # 예시 회사명
            interview_type="general", # 일반 면접 질문
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './data/예시 파일 (마케터).pdf' 파일로 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./data/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. 디지털 마케팅 스터디와 뷰티 에디터 서포터즈 활동에서 배운 점은 무엇이며, 그것이 ABC 마케팅의 프로젝트에 어떻게 적용될 수 있을까요?
   가이드라인: 스터디와 서포터즈 활동을 통해 얻은 실질적인 경험과 교훈을 구체적으로 설명하고, 이를 어떻게 ABC 마케팅의 프로젝트나 문화에 기여할 수 있을지 연결 지어 설명하세요.
2. 개인 뷰티 채널 운영 경험을 통해 배운 가장 중요한 마케팅 전략은 무엇이었나요?
   가이드라인: 유튜브와 인스타그램을 통한 채널 운영 경험에서 얻은 주요 전략이나 인사이트를 강조하고, 구체적인 사례와 성과를 통해 설명하세요.
3. 뷰티 트렌드를 분석하고 이를 콘텐츠로 어떻게 빠르게 구현하였는지 설명해 주세요.
   가이드라인: 최신 뷰티 트렌드를 파악하는 방법과 이러한 트렌드를 콘텐츠로 빠르게 반영한 구체적인 사례를 제시하세요.
4. ABC 마케팅의 문화와 비전과 관련하여 당신이 가장 잘 적합하다고 생각하는 이유는 무엇인가요?
   가이드라인: ABC 마케팅의 문화와 비전을 사전에 조사하고, 개인의 가치관과 경험이 어떻게 그것들과 일치하는지 설명하세요.
5. 데이터 기반의 콘텐츠 성과 분석에서 가장 중요하게 여기는 지표는 무엇이며, 이를 통해 어떤 개선을 이루었나요?
   가이드라인: 마케팅 성과 지표 중 가장 중요한 것을 선택하고, 이를 통해 이루어진 개선 사례를 구체적으로 설명하세요.
6. 소셜 미디어 플랫폼별로 콘텐츠 기획 시 가장 중요하게 고려하는 요소는 무엇인가요?
   가이드라인: 각 플랫폼의 특성과 사용자 행동을 고려한 콘텐츠 기획의 차별점과 전략을 설명하세요.
7. 비건 뷰티 브랜드 론칭 전략 공모전에서 배운 점이 ABC 마케팅에서

In [ ]:
# 제공하는 PDF 파일로 예상 면접 질문 생성 - 개발자 버전
resume_file_path = "./resume/예시파일 (개발자).pdf"

# 파일 존재 여부 확인 (오류 방지용)
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성
        # 필요에 따라 company_name을 추가할 수 있습니다.
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="삼성", # 예시 회사명
            interview_type="technical", # 일반 면접 질문
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './data/예시파일 (개발자).pdf' 파일로 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./data/예시파일 (개발자).pdf
총 3개 문서/페이지 로드됨
총 5개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. Spring Boot와 JPA를 활용한 프로젝트 경험에 대해 설명해주시고, 특히 API 서버 구축 시 어떤 점을 중점적으로 고려했는지 말씀해 주세요.
   가이드라인: 프로젝트의 목표와 성과를 명확히 설명하고, API 설계 시 성능 최적화나 데이터 일관성 유지 등 기술적 고려 사항을 강조하세요.
2. Kafka 기반의 이벤트 드리븐 아키텍처를 도입한 경험이 있으신데, 삼성의 대규모 데이터 처리 시스템에 어떻게 적용할 수 있을까요?
   가이드라인: Kafka를 활용한 경험을 바탕으로 대규모 시스템의 데이터 일관성과 확장성을 강조하고, 삼성의 제품에 맞춘 구체적인 적용 방안을 제시하세요.
3. Docker와 Kubernetes를 활용한 배포 시간 단축 경험을 설명하시고, 이 기술이 삼성의 클라우드 서비스에 어떻게 기여할 수 있는지 설명해주세요.
   가이드라인: Docker와 Kubernetes의 장점을 기반으로 효율적인 배포 및 운영 관리 경험을 설명하고, 삼성의 클라우드 서비스와의 시너지를 강조하세요.
4. Redis를 활용한 캐싱 전략으로 성능을 개선한 사례를 설명해 주시고, 삼성의 시스템에서 어떻게 활용할 수 있을지 논의해 주세요.
   가이드라인: Redis를 통해 성능을 개선한 구체적인 사례를 설명하고, 삼성의 시스템에 적합한 성능 개선 전략을 제안하세요.
5. Git을 통한 협업 경험과 코드 리뷰의 중요성을 어떻게 체감하셨는지, 삼성의 팀 환경에서 어떻게 적용할 수 있을지 설명해 주세요.
   가이드라인: 협업의 중요성과 Git을 활용한 구체적인 사례를 설명하고, 삼성의 팀 협업 문화에 기여할 수 있는 방법을 제시하세요.
6. 캡스톤 디자인 프로젝트에서 딥러닝 기

In [ ]:
# 희망 직무 추가 버전
resume_file_path = "./resume/예시 파일 (마케터).pdf"

if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 희망 직무 '디지털 마케터' 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성 (희망 직무 추가)
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="ABC 회사", # 예시 회사명
            interview_type="general", # 일반 면접 질문
            desired_job_role="디지털 마케터", # 희망 직무 입력
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './data/예시 파일 (마케터).pdf' 파일로 희망 직무 '디지털 마케터' 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./data/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. 디지털 마케팅 실전 스터디에서 배운 내용 중 ABC 회사에서 어떻게 적용할 수 있을지를 설명해 주실 수 있나요?
   가이드라인: 스터디에서 배운 퍼포먼스 마케팅과 콘텐츠 마케팅 전략을 ABC 회사의 비전이나 제품과 연결하여 설명하세요.
2. 뷰티 산업에서의 경험이 ABC 회사의 디지털 마케팅 전략에 어떻게 기여할 수 있을까요?
   가이드라인: 뷰티 산업에 대한 깊은 이해를 바탕으로 소비자 니즈와 트렌드를 어떻게 분석하고, 이를 ABC 회사의 마케팅 전략에 통합할 수 있을지 설명하세요.
3. 개인 뷰티 채널 '수진's 뷰티로그' 운영 경험이 디지털 마케터로서 당신의 역량에 어떻게 기여했나요?
   가이드라인: 채널 운영을 통해 얻은 콘텐츠 기획, 분석, 문제 해결 능력 등을 강조하고, 이 경험이 디지털 마케팅 업무에 어떻게 활용될 수 있는지 설명하세요.
4. 뷰티 에디터 서포터즈 활동에서 가장 기억에 남는 성과는 무엇이며, 그 경험으로부터 배운 점은 무엇인가요?
   가이드라인: 특정 캠페인이나 콘텐츠의 성과와 그로부터 얻은 인사이트를 구체적으로 설명하고, 이를 통해 배운 점을 강조하세요.
5. ABC 회사는 어떤 문화와 비전을 가지고 있다고 생각하며, 그에 맞춰 본인의 강점을 어떻게 활용할 수 있을까요?
   가이드라인: ABC 회사의 문화와 비전에 대한 이해를 바탕으로 자신의 강점과 경험이 어떻게 기여할 수 있는지 설명하세요.
6. 당신의 데이터 분석 능력은 어떤 방식으로 마케팅 성과를 높이는 데 기여할 수 있을까요?
   가이드라인: 구체적인 데이터 분석 도구 사용 경험과 이를 통해 얻은 마케팅 성과 개선 사례를 설명하세요.
7. 다양

### 이력서 파일 참고하여 추천 직무, 역량 추천

In [13]:
# 추천 직무 및 필요 역량 생성

# 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict, Optional

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser # JSON 형식으로 출력 파싱

# API KEY 확인
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")

OPENAI_API_KEY: sk


In [14]:
# 이력서 로딩 및 벡터 저장소 생성 함수
def load_resume_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    이력서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    지원 형식: PDF, TXT
    """
    try:
        print(f"이력서 파일 로딩 중: {file_path}")

        # 파일 확장자에 따라 적절한 로더 사용
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"이력서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e


In [15]:
# 추천 직무 및 필요 역량 생성 로직
def recommend_job_and_skills(
    vectorstore,
    temperature: float = 0.5,
    num_recommendations: int = 3
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)를 기반으로 추천 직무와 해당 직무에 필요한 역량을 생성합니다.
    생성된 결과는 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 커리어 컨설턴트이며, 제공된 이력서 내용을 바탕으로 지원자에게 가장 적합한 직무와 해당 직무에 필요한 핵심 역량을 추천해야 합니다.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 관심사 등을 파악하세요.
        2.  지원자의 역량과 경험에 가장 부합하는 {num_recommendations}개의 직무를 추천해주세요.
        3.  각 추천 직무에 대해 해당 직무에서 성공하기 위해 필요한 핵심 역량(기술 및 소프트 스킬)을 3~5가지 나열해주세요.
        4.  전반적인 커리어 개발을 위한 조언을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "recommended_roles": [
            {{{{
            "role_name": "추천 직무명 1",
            "required_skills": ["필요 역량 1", "필요 역량 2", "필요 역량 3"]
            }}}},
            {{{{
            "role_name": "추천 직무명 2",
            "required_skills": ["필요 역량 A", "필요 역량 B", "필요 역량 C"]
            }}}}
        ],
        "overall_career_advice": "전반적인 커리어 개발 조언 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 직무와 역량을 추천할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "이력서 기반 직무 및 역량 추천을 해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"직무 및 역량 추천 중 오류 발생: {str(e)}")
        return {"error": f"직무 및 역량 추천 중 오류가 발생했습니다: {str(e)}"}


In [ ]:
#  PDF 파일로 추천 직무 및 필요 역량 생성 - 마케터 버전

resume_file_path = "./resume/예시 파일 (마케터).pdf"

# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 추천 직무 및 필요 역량 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 추천 직무 및 필요 역량 생성
        recommendations_marketer = recommend_job_and_skills(
            vectorstore_marketer,
            temperature=0.6,
            num_recommendations=3
        )

        # 3. 결과 출력
        if "error" in recommendations_marketer:
            print(f"오류: {recommendations_marketer['error']}")
        else:
            print("\n생성된 추천 직무 및 필요 역량:")
            for i, role in enumerate(recommendations_marketer['recommended_roles']):
                print(f"{i+1}. 직무명: {role['role_name']}")
                print(f"   필요 역량: {', '.join(role['required_skills'])}")
            print(f"\n전반적인 커리어 조언: {recommendations_marketer['overall_career_advice']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './data/예시 파일 (마케터).pdf' 파일로 추천 직무 및 필요 역량 생성 ---
이력서 파일 로딩 중: ./data/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 추천 직무 및 필요 역량:
1. 직무명: 디지털 마케팅 전문가
   필요 역량: 소셜 미디어 콘텐츠 기획 및 제작, SEO 및 데이터 분석 능력, 인플루언서 및 바이럴 마케팅 전략 수립
2. 직무명: 콘텐츠 마케팅 매니저
   필요 역량: 스토리텔링 및 카피라이팅, 뷰티 트렌드 분석 및 소비자 니즈 파악, 영상 편집 및 콘텐츠 성과 분석
3. 직무명: 뷰티 브랜드 마케팅 전략가
   필요 역량: 최신 뷰티 트렌드에 대한 높은 민감성, 경쟁사 및 소비자 행동 분석, 마케팅 성과 지표 이해 및 활용

전반적인 커리어 조언: 지원자는 디지털 및 콘텐츠 마케팅 분야에서 강점을 가지고 있으며, 최신 뷰티 트렌드에 대한 높은 이해도를 바탕으로 뷰티 업계의 마케팅 전문가로 성장할 수 있습니다. 다양한 프로젝트 경험을 바탕으로 지속적인 성과 분석과 개선을 통해 역량을 더욱 향상시키세요.


In [ ]:
#  PDF 파일로 추천 직무 및 필요 역량 생성 - 개발자 버전

resume_file_path = "./resume/예시파일 (개발자).pdf"

# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 추천 직무 및 필요 역량 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 추천 직무 및 필요 역량 생성
        recommendations_marketer = recommend_job_and_skills(
            vectorstore_marketer,
            temperature=0.6,
            num_recommendations=3
        )

        # 3. 결과 출력
        if "error" in recommendations_marketer:
            print(f"오류: {recommendations_marketer['error']}")
        else:
            print("\n생성된 추천 직무 및 필요 역량:")
            for i, role in enumerate(recommendations_marketer['recommended_roles']):
                print(f"{i+1}. 직무명: {role['role_name']}")
                print(f"   필요 역량: {', '.join(role['required_skills'])}")
            print(f"\n전반적인 커리어 조언: {recommendations_marketer['overall_career_advice']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './data/예시파일 (개발자).pdf' 파일로 추천 직무 및 필요 역량 생성 ---
이력서 파일 로딩 중: ./data/예시파일 (개발자).pdf
총 3개 문서/페이지 로드됨
총 5개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 추천 직무 및 필요 역량:
1. 직무명: DevOps Engineer
   필요 역량: Docker 및 Kubernetes를 활용한 컨테이너화 및 오케스트레이션, CI/CD 파이프라인 구축 및 관리, AWS 클라우드 서비스 활용 능력, 문제 해결 및 성능 최적화 능력, 협업 및 커뮤니케이션 스킬
2. 직무명: Backend Developer
   필요 역량: Java 및 Spring Boot 기반의 백엔드 개발, RESTful API 설계 및 구현, 데이터베이스 설계 및 최적화, 비동기 처리 및 대용량 데이터 처리, 테스트 주도 개발(TDD) 및 코드 품질 관리
3. 직무명: Cloud Solutions Architect
   필요 역량: 클라우드 인프라 설계 및 최적화, 마이크로서비스 아키텍처 설계, Kafka를 활용한 이벤트 드리븐 아키텍처, Redis 및 MySQL 등 데이터베이스 관리, 기술 블로그 운영 및 기술 문서 작성 능력

전반적인 커리어 조언: 현재의 강점을 살려 DevOps 및 클라우드 아키텍처 분야로의 전문성을 강화하세요. 최신 기술 동향을 지속적으로 학습하고, 오픈소스 프로젝트 기여를 통해 네트워크를 확장하세요.


### 자소서 파일을 받아서 자소서 업그레이드